<a href="https://colab.research.google.com/github/YoshidaR3n/YoshidaR3n/blob/main/discord_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U discord.py[voice]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.6 MB/s eta 0:00:00


In [ ]:
pip install google-api-python-client

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 11.8 MB/s eta 0:00:00


## youtubeAPIで動画の情報を取得

In [1]:
from apiclient.discovery import build   #ライブラリをインポート
from google.colab import userdata


# youtubeのAPI設定
YOUTUBE_API_KEY = userdata.get('YOUTUBE_API_KEY')
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(
    YOUTUBE_API_SERVICE_NAME,
    YOUTUBE_API_VERSION,
    developerKey=YOUTUBE_API_KEY
)

In [ ]:
#動画の情報をとってくる
def get_video_info(keyword):
    # search settings
    youtube_query = youtube.search().list(
        part='id,snippet',
        q=keyword,
        type='video',
        maxResults=1,
        order='relevance',
    )

    # execute()で検索を実行
    youtube_response = youtube_query.execute()

    # 検索結果を取得し、リターンする
    return youtube_response.get('items', [])

## gemini使用準備

In [ ]:
#参考：https://note.com/npaka/n/n166bc3df3abc
import pathlib
import textwrap
import google.generativeai as genai


# geminiを使うためのAPIKEY設定
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# モデルの準備
model = genai.GenerativeModel('gemini-pro')

# モデルの読み込み
model_v = genai.GenerativeModel('gemini-pro-vision')

## discord上で送信された画像をダウンロードする関数


In [ ]:
import urllib.request
import imghdr
import PIL.Image

def tenpu_image_download(url, save_pass):
    #添付画像をダウンロード
    opener = urllib.request.build_opener()
    opener.addheaders = [("User-agent", "Mozilla/5.0")]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(url, save_pass)
    return

def download_image_class(message, save_pass):
  url = message.attachments[0].url
  tenpu_image_download(url, save_pass)
  # jpgに変換して上書き保存
  imagetype = imghdr.what(save_pass)

  if imagetype == "png":
    image_convert = PIL.Image.open(save_pass)
    image_convert = image_convert.convert("RGB")
    image_convert.save(save_pass)

## BOT本体の挙動を設定

In [ ]:
# インストールした discord.py を読み込む
import discord
import re #正規表現のためのライブラリ
import random
import requests
import json

save_pass = "/content/drive/MyDrive/tmp.jpg"

TOKEN = userdata.get('BOT_TOKEN')

# 接続に必要なオブジェクトを生成
client = discord.Client(intents=discord.Intents.all())

# 起動時に動作する処理
@client.event
async def on_ready():
    # 起動したらターミナルにログイン通知が表示される
    print('ログインしました')

# メッセージ受信時に動作する処理
@client.event
async def on_message(message):
  # メッセージ送信者がBotだった場合は無視する
     if message.author.bot:
         return

    # 「!!」から始まる文字列がマッチしなかったら無視する
     if message.content:
      if re.match("/ai.*", message.content) != None:
        zyumon = str("次の質問に答えてください。" + message.content[4:])
        ai_response = model.generate_content(zyumon)
        response = str(ai_response.text)
        await message.channel.send(response)

      if re.match("/im.*", message.content) != None:
        download_image_class(message, save_pass)
        img = PIL.Image.open("/content/drive/MyDrive/tmp.jpg")
        ai_response = model_v.generate_content(["この写真について説明して",img])
        response = ai_response.text
        await message.channel.send(response)

      if re.match("/we .*", message.content) != None:
        res = requests.get("https://api.openweathermap.org/data/2.5/weather?q={}&appid=b4a403d67ab7707a58bebf75527a4063&units=metric&lang=ja".format(message.content[4:]))
        res_dict = json.loads(res.text)
        data = str("天気："+res_dict["weather"][0]["description"]+"\nただ今の気温："+str(res_dict["main"]["temp"])+"度")
        await message.channel.send(data)

      if re.match("/yo .*", message.content) == None:
        return

      data = get_video_info(message.content[3:])
      for video in data:
        result = str(video['snippet']['title']) + " URL=>https://www.youtube.com/watch?v={}".format(video['id']['videoId'])
        await message.channel.send(result)

## 起動

In [ ]:
import nest_asyncio

nest_asyncio.apply()
# Botの起動とDiscordサーバーへの接続
client.run(TOKEN)

2024-07-09 02:22:58 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2024-07-09 02:22:59 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 19f2963eac152586d843351e7e0d9cef).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 19f2963eac152586d843351e7e0d9cef).


ログインしました


2024-07-09 02:25:47 ERROR    discord.client Ignoring exception in on_message
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/discord/client.py", line 449, in _run_event
    await coro(*args, **kwargs)
  File "<ipython-input-8-67a2cff38fee>", line 47, in on_message
    data = str("天気："+res_dict["weather"][0]["description"]+"\nただ今の気温："+str(res_dict["main"]["temp"])+"度")
KeyError: 'weather'
ERROR:discord.client:Ignoring exception in on_message
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/discord/client.py", line 449, in _run_event
    await coro(*args, **kwargs)
  File "<ipython-input-8-67a2cff38fee>", line 47, in on_message
    data = str("天気："+res_dict["weather"][0]["description"]+"\nただ今の気温："+str(res_dict["main"]["temp"])+"度")
KeyError: 'weather'
